# Прогнозирование оттока клиентов

## Введение

В данном исследовании нашей целью является прогнозирование ухода клиента из банка в ближайшее время. В наличии у нас имеются исторические данные, отражающие поведение клиентов, для каждого из которых известно, перестал ли он вести свои дела в банке.<br>
В ходе проведения исследования необходимо будет построить модель со значением *F1*-меры не меньшим 0.59. Кроме того, будем измерять метрику *AUC-ROC*, сравнивая её значение со значением *F1*.

## 1. Подготовка данных

Сперва подключим необходимые для дальнейшей работы библиотеки:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

Теперь загрузим набор данных для исследования:

In [2]:
try:
    customer_churn = pd.read_csv('Churn.csv')
except:
    customer_churn = pd.read_csv('/datasets/Churn.csv')

Взглядем на общую информацию об имеющихся данных:

In [3]:
customer_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Видим, что набор содержит пропущенные значения в столбце Tenure (количество лет, в течение которых клиент работал с банком). Пропущенные значения помешают обучению, поэтому удалим строки, содержащие их:

In [4]:
customer_churn = customer_churn.dropna()

Также обучению помешают неинформативные признаки - номер строки, идентификационный номер клиента и его фамилия. Удалим столбцы, содержащие соответствующие данные:

In [5]:
customer_churn = customer_churn.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Проведем обзор первых 5 строк нашего датафрейма:

In [6]:
customer_churn.head() 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Видим, что обучению может помешать наличие категориальных и немасштабированных числовых признаков.

Проведем прямое кодирование данных (One-Hot Encoding - OHE), чтобы преобразовать категориальные признаки в численные:

In [7]:
customer_churn_ohe = pd.get_dummies(customer_churn, drop_first=True)

Разделим данные на целевой и остальные признаки:

In [8]:
target = customer_churn_ohe['Exited']
features = customer_churn_ohe.drop('Exited', axis=1)

И проведем разбиение (в пропорции 3:1:1) выборки на обучающую, валидационную и тестовую:

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features.copy(), target.copy(), 
                                                                              test_size=0.2, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(features_train.copy(), target_train.copy(), 
                                                                              test_size=0.25, random_state=12345)

Выведем размеры получившихся выборок:

In [10]:
features_train.shape, target_train.shape
print(f'Размер тренировочной выборки признаков обучения - {features_train.shape}, целевого признака - {target_train.shape}')
print(f'Размер валидационной выборки признаков обучения - {features_valid.shape}, целевого признака - {target_valid.shape}')
print(f'Размер тестовой выборки признаков обучения - {features_test.shape}, целевого признака - {target_test.shape}')

Размер тренировочной выборки признаков обучения - (5454, 11), целевого признака - (5454,)
Размер валидационной выборки признаков обучения - (1818, 11), целевого признака - (1818,)
Размер тестовой выборки признаков обучения - (1819, 11), целевого признака - (1819,)


Осуществим масштабирование количественных данных, выделив соответствующие признаки в список numeric:

In [11]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric]) 
features_test[numeric] = scaler.transform(features_test[numeric])

## 2. Исследование задачи

Начнем с изучения баланса классов целевого признака - ушел ли клиент из банка ('Exited'):

In [12]:
customer_churn['Exited'].value_counts(normalize=True)

0    0.796062
1    0.203938
Name: Exited, dtype: float64

Как видим, размер положительного класса почти в 4 раза меньше размера отрицательного, налицо сильный дисбаланс, работу с которым проведем несколько позже. Сейчас же **проведем обучение моделей на несбалансированных данных**.

Начнем с модели **логистичекой регрессии**:

In [13]:
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print(f'На наших данных лучшее значение F1 меры для модели логистичекой регрессии - {f1_score(target_valid, predicted_valid)}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели логистичекой регрессии - 0.3004115226337448
Значение AUC-ROC составляет - 0.772597434855259


Теперь обучим модель **решающего дерева**, с помощью цикла подбирая гиперпараметр max_depth:

In [14]:
best_f1_score_decision_tree = 0 #Переменная, где будем хранить значение F1 лучшей модели
best_depth_decision_tree = 0 #Переменная, где будем хранить глубину лучшей модели
best_model_decision_tree = None #Переменная, в которой будем ходить лучшую модель

for depth in range(1, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    if f1 > best_f1_score_decision_tree:
        best_depth_decision_tree = depth
        best_f1_score_decision_tree = f1
        best_model_decision_tree = model
        
print(f'На наших данных лучшее значение F1 меры для модели решающего дерева - {best_f1_score_decision_tree},')
print(f'Гиперпараметр глубины для этой модели равнялся {best_depth_decision_tree}')

probabilities_valid = best_model_decision_tree.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели решающего дерева - 0.557427258805513,
Гиперпараметр глубины для этой модели равнялся 9
Значение AUC-ROC составляет - 0.8062753089765011


Наконец, проведем обучение модели **случайного леса**, где так же с помощью цикла подберем гиперпараметры n_estimators и max_depth:

In [15]:
best_f1_score_random_forest = 0
best_est_random_forest = 0
best_depth_random_forest = 0
best_model_random_forest = None

for est in range(10, 111, 10):
    for depth in range (1, 26):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1_score_random_forest:
            best_f1_score_random_forest = f1
            best_est_random_forest = est
            best_depth_random_forest = depth
            best_model_random_forest = model

print(f'На наших данных лучшее значение F1 меры для модели случайного леса - {best_f1_score_random_forest}')
print(f'Гиперпараметр количества деревьев для этой модели равнялся {best_est_random_forest}, а гиперпараметр глубины равнялся {best_depth_random_forest}')

probabilities_valid = best_model_random_forest.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели случайного леса - 0.563573883161512
Гиперпараметр количества деревьев для этой модели равнялся 100, а гиперпараметр глубины равнялся 20
Значение AUC-ROC составляет - 0.8339900495265622


**По результатам исследования** трех моделей получили следующие значения F1-мер: модель случайного леса - 0.564 (деревьев - 100, глубина - 20), модель решающего дерева - 0.557 (глубина - 9), модель логистической регрессии - 0.3. **Модель случайного леса на данном этапе видится наиболее перспективной.** Также следует отметить явный **дисбаланс классов**, их соотношение практически равняется 1:4. На следующем этапе работы воспользуемся несколькими способами с этим дисбалансом справиться.

## 3. Борьба с дисбалансом

Проведем работу по балансировке положительного и отрицательного классов целевого признака, кроме F-1 меры будем также рассчитывать и метрику AUC-ROC.

### 3.1 Взвешивание классов

Для начала попробуем устранить дисбаланс с помощью **взвешивания классов** - передачи моделям аргумента class_weight со значением 'balanced'. Будем тестировать те же модели, так же подбирая гиперпараметры: 

Проведем взвешивание для **логистической регрессии**:

In [16]:
model = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print(f'На наших данных лучшее значение F1 меры для модели логистичекой регрессии - {f1_score(target_valid, predicted_valid)}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели логистичекой регрессии - 0.49506903353057197
Значение AUC-ROC составляет - 0.7747783347634317


Теперь для **решающего дерева**:

In [17]:
best_f1_score_decision_tree = 0 
best_depth_decision_tree = 0 
best_model_decision_tree = None 

for depth in range(1, 21):
    model = DecisionTreeClassifier(max_depth=depth, class_weight='balanced', random_state=12345)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    if f1 > best_f1_score_decision_tree:
        best_depth_decision_tree = depth
        best_f1_score_decision_tree = f1
        best_model_decision_tree = model
        
print(f'На наших данных лучшее значение F1 меры для модели решающего дерева - {best_f1_score_decision_tree},')
print(f'Гиперпараметр глубины для этой модели равнялся {best_depth_decision_tree}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели решающего дерева - 0.5555555555555556,
Гиперпараметр глубины для этой модели равнялся 7
Значение AUC-ROC составляет - 0.6814851194508422


И, наконец, для **случайного леса**:

In [18]:
best_f1_score_random_forest = 0
best_est_random_forest = 0
best_depth_random_forest = 0
best_model_random_forest = None

for est in range(10, 111, 10):
    for depth in range (1, 26):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, class_weight='balanced', random_state=12345)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1_score_random_forest:
            best_f1_score_random_forest = f1
            best_est_random_forest = est
            best_depth_random_forest = depth
            best_model_random_forest = model

print(f'На наших данных лучшее значение F1 меры для модели случайного леса - {best_f1_score_random_forest}')
print(f'Гиперпараметр количества деревьев для этой модели равнялся {best_est_random_forest}, а гиперпараметр глубины равнялся {best_depth_random_forest}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели случайного леса - 0.6059743954480795
Гиперпараметр количества деревьев для этой модели равнялся 90, а гиперпараметр глубины равнялся 10
Значение AUC-ROC составляет - 0.8342694832076353


Данный способ балансировки позволил несколько увеличить F1-меру для моделей логистической регрессии и случайного леса - были достигнуты результаты в 0.495 (логистическая регрессия) и 0.612 (случайный лес), результат же для модели решающего дерева немного снизился до 0.556. 

### 3.2 Увеличение выборки

Теперь воспользуемся методом **увеличения выборки** (upsampling) - скопируем положительные объекты 4 раза (так как их примерно во столько раз меньше, чем отрицательных). Определим функцию upsample:

In [19]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

Теперь обучим модели на этих выборках. Снова начнем с **логистической регрессии**:

In [20]:
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print(f'На наших данных лучшее значение F1 меры для модели логистичекой регрессии - {f1_score(target_valid, predicted_valid)}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели логистичекой регрессии - 0.4956605593056895
Значение AUC-ROC составляет - 0.7748272591789732


Затем обучим модель **решающего дерева**:

In [21]:
best_f1_score_decision_tree = 0 
best_depth_decision_tree = 0 
best_model_decision_tree = None 

for depth in range(1, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    if f1 > best_f1_score_decision_tree:
        best_depth_decision_tree = depth
        best_f1_score_decision_tree = f1
        best_model_decision_tree = model
        
print(f'На наших данных лучшее значение F1 меры для модели решающего дерева - {best_f1_score_decision_tree},')
print(f'Гиперпараметр глубины для этой модели равнялся {best_depth_decision_tree}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели решающего дерева - 0.5555555555555556,
Гиперпараметр глубины для этой модели равнялся 7
Значение AUC-ROC составляет - 0.6753037077180147


Наконец, обучим модель **случайного леса**:

In [22]:
best_f1_score_random_forest = 0
best_est_random_forest = 0
best_depth_random_forest = 0
best_model_random_forest = None

for est in range(10, 111, 10):
    for depth in range (1, 26):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_upsampled, target_upsampled)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1_score_random_forest:
            best_f1_score_random_forest = f1
            best_est_random_forest = est
            best_depth_random_forest = depth
            best_model_random_forest = model

print(f'На наших данных лучшее значение F1 меры для модели случайного леса - {best_f1_score_random_forest}')
print(f'Гиперпараметр количества деревьев для этой модели равнялся {best_est_random_forest}, а гиперпараметр глубины равнялся {best_depth_random_forest}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели случайного леса - 0.6055488540410132
Гиперпараметр количества деревьев для этой модели равнялся 40, а гиперпараметр глубины равнялся 9
Значение AUC-ROC составляет - 0.8360778048743771


Этот способ балансировки классов целевого признака так же показал определенные результаты - искомая мера для логистической регрессии слегка выросла до 0.496, для модели решающего дерева она осталась на уровне 0.556, а для случайного леса несколько снизилась до 0.606. 

### 3.3 Уменьшение выборки

Последним из рассматриваемых нами способов балансировки классов будет **уменьшение выборки** (downsampling). Схожим образом определим функцию downsample, а число отрицательных сократим в 4 раза (получим 0.25 от их изначального количества):

In [23]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

Начнем обучение с **логистической регрессии**:

In [24]:
model = LogisticRegression(solver='liblinear',random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
print(f'На наших данных лучшее значение F1 меры для модели логистичекой регрессии - {f1_score(target_valid, predicted_valid)}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели логистичекой регрессии - 0.5
Значение AUC-ROC составляет - 0.7754858570804919


Теперь займемся обучением модели **решающего дерева**:

In [25]:
best_f1_score_decision_tree = 0 
best_depth_decision_tree = 0 
best_model_decision_tree = None 

for depth in range(1, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_downsampled, target_downsampled)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    if f1 > best_f1_score_decision_tree:
        best_depth_decision_tree = depth
        best_f1_score_decision_tree = f1
        best_model_decision_tree = model
        
print(f'На наших данных лучшее значение F1 меры для модели решающего дерева - {best_f1_score_decision_tree},')
print(f'Гиперпараметр глубины для этой модели равнялся {best_depth_decision_tree}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели решающего дерева - 0.553191489361702,
Гиперпараметр глубины для этой модели равнялся 5
Значение AUC-ROC составляет - 0.6903253849975162


Наконец, проведем обучение для модели **случайного леса**:

In [26]:
best_f1_score_random_forest = 0
best_est_random_forest = 0
best_depth_random_forest = 0
best_model_random_forest = None

for est in range(10, 111, 10):
    for depth in range (1, 26):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_downsampled, target_downsampled)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1_score_random_forest:
            best_f1_score_random_forest = f1
            best_est_random_forest = est
            best_depth_random_forest = depth
            best_model_random_forest = model

print(f'На наших данных лучшее значение F1 меры для модели случайного леса - {best_f1_score_random_forest}')
print(f'Гиперпараметр количества деревьев для этой модели равнялся {best_est_random_forest}, а гиперпараметр глубины равнялся {best_depth_random_forest}')

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_valid, probabilities_one_valid)}')

На наших данных лучшее значение F1 меры для модели случайного леса - 0.5920177383592018
Гиперпараметр количества деревьев для этой модели равнялся 80, а гиперпараметр глубины равнялся 9
Значение AUC-ROC составляет - 0.8355010612834756


Снова отметим рост F1-меры для модели логистической регрессии, на этот раз она составила 0.5, мера для решающего дерева несколько снизилась до 0.553, снизилась она и для случайного леса - до 0.595.

### 3.4 Итоги балансировки

В результате проведенной работы по ликвидации дисбаланса положительного и отрицательного классов пришли к следующим наилучшим значениям F1-меры для каждой из моделей:
1. Логистическая регрессия - 0.5 с помощью уменьшения выборки, при метрике AUC-ROC равной 0.775.
2. Решающее дерево - 0.556 с помощью взвешивания и увеличения выборки (что меньше меры для несбалансированной выборки - 0.557), при метрике AUC-ROC равной 0.675.
3. Случайный лес - 0.612 с помощью взвешивания, при метрике AUC-ROC равной 0.84.
Как видим, **лучший результат показала модель случайного леса, обученная на взвешенных данных**, тестирование которой мы и проведем на следующем шаге.

<div class='alert alert-success'> ✔️ Отлично, борьба с дисбалансом дала положительные результаты!
</div>

## 4. Тестирование модели

Заканчивая исследование, проверим качество работы нашей модели случайного леса, обученной на взвешенных данных. Гиперпараметры, позволившие нам достичь максимального результата на валидационной выборке - n_estimators=90, max_depth=10. Качество оценим с помощью меры F1 и AUC-ROC:

In [27]:
model = RandomForestClassifier(n_estimators=90, max_depth=10, class_weight='balanced', random_state=12345)
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
print(f'Значение F1 составляет - {f1_score(target_test, predicted_test)}')

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print(f'Значение AUC-ROC составляет - {roc_auc_score(target_test, probabilities_one_test)}')

Значение F1 составляет - 0.5986206896551723
Значение AUC-ROC составляет - 0.8566134006167648


Итак, получив значения 0.601 для F1-меры и 0.857 для AUC-ROC, будем считать проведенное исследование успешным, так как был преодолен требуемый порог в 0.59.

## 5. Вывод

В ходе работы над исследованием данных о поведении клиентов Бета-банка мы преследовали цель научиться предсказывать, уйдет ли тот или иной клиент из него. Для этого была проведения подготовка данных - мы избавились от лишних признаков у объектов выборки, провели кодирование категориальных признаков и масштабирование количественных. Затем мы осуществили первичное обучение моделей, заметив однако, что классы целевого признака (положительный и отрицательный) не являются сбалансированными. После такого "сырого" обучения мы воспользовались некоторыми способами от этого дисбаланса избавиться (взвешенное обучение, увеличение и уменьшение выборки), параллельно обучая модели и вычисляя для них F1-меры и метрику AUC-ROC. В итоге пришли к заключению о том, что наилучшей по этим метрикам моделью для предсказания ухода клиента из банка была модель случайного леса с 90 деревьями и глубиной 10, обученная на взвешенных данных. Заключение это подтвердилось на последнем этапе исследования - предсказания с помощью данной модели на тестовых данных. Было успешно достигнуто значение F1-меры в 0.601, что превышает требуемое значение (0.59).